(numpy)=
# Numpy

Широкоиспользуемая библиотека для вычислений с многомерными массивами. API большей частью вдохновлен MATLABом (великая и ужасная среда, язык и IDE для матричных вычислений), а теперь сам является примером для подражания API различных вычислительных пакетов.

Более последовательный гайд стоит посмотреть на [оффсайте библиотеки](https://numpy.org/devdocs/user)


## Массивы

In [1]:
import numpy as np
a = np.array([1, 2, 3]) # создадим вектор
print(a.shape)
b = np.zeros((2, 2)) # создадим матричку 2х2 из нулей
c = np.eye(3) # создадим единичную матрицу 3х3
q = np.random.random((1, 100)) # случайную вектор-строку

(3,)


## Math ops
Для удобства использования np.ndarray определены арифметические операторы, так чтобы соответствовать ожиданиям:

In [2]:
a = np.array([1, 2, 3])
b = np.array([-1, 3, 4])
print(a - b) # разность двух векторов
print(a * b) # поэлементное произведение
print(a@b)  # скалярное произведение


[ 2 -1 -1]
[-1  6 12]
17


## Indexing, slicing and sugar

Numpy поддерживает кажется все разумные варианты индексации:

In [3]:
a = np.arange(16).reshape(4, 4)
# у нас есть массив 4х4 с числами 1..16

# просто по индексам
print("a_{0,1}", a[0, 1], a[0][1])

# по слайсам
print("a_{1,1..3}", a[0, 1:3])
print("a_{2}", a[2], a[2, :], a[2, ...])

# по маске
mask = (a % 3 == 0) # mask.shape == (4, 4)
print(a[mask])

first_rows = np.array([True, True, False, False])
print(a[first_rows])

a_{0,1} 1 1
a_{1,1..3} [1 2]
a_{2} [ 8  9 10 11] [ 8  9 10 11] [ 8  9 10 11]
[ 0  3  6  9 12 15]
[[0 1 2 3]
 [4 5 6 7]]


Для работы с размерностями часто используются еще три конструкции: `None`, `...` (ellipsis, многоточие) и `:` (двоеточие).

In [4]:
# None добавляет ось размерности 1
print(a.shape)
print(a[None].shape)
print(a[:, :, None].shape)

# : превращается в слайс slice(None), берет все элементы вдоль размерности
print(a[2, :])
print(a[2, 0:None])


# ... ellipsis, превращается в необходимое число двоеточий :,:,:

print(a[...], a) # одно и то же

z = np.arange(27).reshape(3, 3, 3)
print(z[0, ..., 1], z[0, :, 1]) # ... удобен когда мы не знаем настоящий шейп массива или нужно не трогать несколько подряд идущих размерностей

(4, 4)
(1, 4, 4)
(4, 4, 1)
[ 8  9 10 11]
[ 8  9 10 11]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]] [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[1 4 7] [1 4 7]


В целом, в numpy очень здорово реализованы методы `__getitem__`/`__setitem__`.

TODO: добавить np.where?


## Broadcasting

Что происходит, если мы хотим арифметику с массивами разных размеров?

In [5]:
a = np.array([1, 2, 3])
k = 2
print(a * k)

[2 4 6]


С точки зрения математики, ничего интересного тут не происходит: мы подразумевали умножение всего вектора на скаляр.
Однако матричные операции в numpy справляются и менее очевидными случаями, например при сложении вектора и скаляра:

In [6]:
a = np.array([1, 2, 3])
k = 2
print(a - k)

[-1  0  1]


В numpy приняты следующие правила работы с массивами разного размера:

1. Размерности сравниваются справа налево
2. Два массива совместимы в размерности, если она одинаковая, либо у одного из массивов единичная.
3. Вдоль отсутствующих размерностей происходит расширение повторением (repeat).

TODO: нарисовать картинку!

Be aware, автоматический броадкастинг легко приводит к ошибкам, так что лучше делать его самостоятельно в явной форме.



## floating point things

np.allclose & dtypes


## numpy & linalg fun

Матричные трюки, вычисление попарных расстояний,
решение СЛУ, обращение матриц, собственные вектора и числа


## Что мы узнали

- основы работы с numpy
- индексацию в массивах
- broadcasting
- floating point things
- numpy fun